In [1]:
%load_ext autoreload
%autoreload 2
import functions as f
from blockchain import Blockchain
from anytree.exporter import DotExporter

In [2]:
bc = Blockchain(max_transactions_per_block = 5, pow_difficulty = 3)

In [3]:
bc.add_account("Alice")
bc.add_account("Bob")

In [4]:
bc.wallets_taking_only_mined_blocks()

{'miner': 100.0}


In [5]:
bc.show()


block  0

header:
nonce  :  2556
block_number  :  0
merkle_tree_root_hash  :  c46ca60643d459f7b73c9bf6fde6ae366f8671bb10e4f3228e8edb77df3f7841
timestamp  :  1655609875982

body:
transactions  :  (('miner', 100.0),)
merkle_tree  :  Node('/c46ca60643d459f7b73c9bf6fde6ae366f8671bb10e4f3228e8edb77df3f7841')
wallets_after_block_transactions  :  {'miner': 100.0}

 ----------------------------------------------------------------------------------------------------

block  1

header:
nonce  :  0
prev_hash  :  00093b1d938f6af714f13b60e7f55be0b14c12f7aa0c368d107ddd0bb4f04a60
block_number  :  1

body:
transactions  :  (('miner', 50.0),)

 ----------------------------------------------------------------------------------------------------


In [6]:
# tx -> (payer,payee,amount_to_transfer,fee)
bc.add_transaction(("miner","Alice",10,1))
bc.add_transaction(("miner","Bob",10,1))

In [7]:
bc.mine_last_block()

In [8]:
bc.show()


block  0

header:
nonce  :  2556
block_number  :  0
merkle_tree_root_hash  :  c46ca60643d459f7b73c9bf6fde6ae366f8671bb10e4f3228e8edb77df3f7841
timestamp  :  1655609875982

body:
transactions  :  (('miner', 100.0),)
merkle_tree  :  Node('/c46ca60643d459f7b73c9bf6fde6ae366f8671bb10e4f3228e8edb77df3f7841')
wallets_after_block_transactions  :  {'miner': 100.0}

 ----------------------------------------------------------------------------------------------------

block  1

header:
nonce  :  946
prev_hash  :  00093b1d938f6af714f13b60e7f55be0b14c12f7aa0c368d107ddd0bb4f04a60
block_number  :  1
merkle_tree_root_hash  :  4bf012f4a859b4da71bbf55ddfc8f4b1fbc01a53b4b379e43bb467a377f9f0c3
timestamp  :  1655609881798

body:
transactions  :  (('miner', 50.0), ('miner', 'Alice', 10), ('miner', 'miner', 1), ('miner', 'Bob', 10), ('miner', 'miner', 1))
merkle_tree  :  Node('/4bf012f4a859b4da71bbf55ddfc8f4b1fbc01a53b4b379e43bb467a377f9f0c3')
wallets_after_block_transactions  :  {'miner': 130.0, 'Alice': 

In [10]:
# visualize last mined block
bc.visualize_merkle_tree(number_block = -2)

4bf012f4a859b4da71bbf55ddfc8f4b1fbc01a53b4b379e43bb467a377f9f0c3
├── 45ec05d1f448ba5241da2402bfcd9b15f47994a45e36ad00f66550297fe2fe6e
│   └── d6fce19356c28302e073cfd0214f5e19010224d476360e693f651dfdac976dbb
│       └── bb0ee4d3786b6a31864a3dfb1d50a8f2c2f70cadd12c013a80f75c85ee8e28e7
│           └── ('miner', 50.0)
└── 570490d5aff85d1399e4ccedb2d4fa2a2649e5f521975e5d2614d04b1f18bcc7
    ├── c65c087034fb725599502e1c773e14cddc7c3cc73b7d82c38016ed18e2e15e1b
    │   ├── b4aabe4d189f69d4ce43528d2db383d0d95edca87fe71c05174a9abfdbcf9333
    │   │   └── ('miner', 'Alice', 10)
    │   └── 19be4cfe23fb44f0bb679d993561ff582d010775a264d53a59bdf0c42eec785d
    │       └── ('miner', 'miner', 1)
    └── 386cf0679672f5c569191753bb9fddd4648396b26c625d362caf9830f7422876
        ├── 9eb06e876613f878a7bf6d8e03e225fd0c04299ed7fc3da7543c72be913963f0
        │   └── ('miner', 'Bob', 10)
        └── 19be4cfe23fb44f0bb679d993561ff582d010775a264d53a59bdf0c42eec785d
            └── ('miner', 'miner', 1)


In [11]:
f.string_to_hash(str((bc.chain[0]["header"])))

'00093b1d938f6af714f13b60e7f55be0b14c12f7aa0c368d107ddd0bb4f04a60'

In [12]:
idx = 1
f.string_to_hash(str((bc.chain[idx]["header"]))) == bc.chain[idx+1]["header"]["prev_hash"]

True

In [13]:
bc.wallets_taking_only_mined_blocks()

{'miner': 130.0, 'Alice': 10, 'Bob': 10}


In [14]:
bc.add_transaction(("Bob","Bob",5,1))

In [15]:
bc.mine_last_block()

In [16]:
bc.show()


block  0

header:
nonce  :  2556
block_number  :  0
merkle_tree_root_hash  :  c46ca60643d459f7b73c9bf6fde6ae366f8671bb10e4f3228e8edb77df3f7841
timestamp  :  1655609875982

body:
transactions  :  (('miner', 100.0),)
merkle_tree  :  Node('/c46ca60643d459f7b73c9bf6fde6ae366f8671bb10e4f3228e8edb77df3f7841')
wallets_after_block_transactions  :  {'miner': 100.0}

 ----------------------------------------------------------------------------------------------------

block  1

header:
nonce  :  946
prev_hash  :  00093b1d938f6af714f13b60e7f55be0b14c12f7aa0c368d107ddd0bb4f04a60
block_number  :  1
merkle_tree_root_hash  :  4bf012f4a859b4da71bbf55ddfc8f4b1fbc01a53b4b379e43bb467a377f9f0c3
timestamp  :  1655609881798

body:
transactions  :  (('miner', 50.0), ('miner', 'Alice', 10), ('miner', 'miner', 1), ('miner', 'Bob', 10), ('miner', 'miner', 1))
merkle_tree  :  Node('/4bf012f4a859b4da71bbf55ddfc8f4b1fbc01a53b4b379e43bb467a377f9f0c3')
wallets_after_block_transactions  :  {'miner': 130.0, 'Alice': 

In [17]:
bc.visualize_merkle_tree(number_block = -2)

32c07cbdfc7eeb47dd7c94e8d90afade98d39c24dedfa4f11f2137fd6d4cf287
├── 7921d5246cf35f6bc33376028ea662626a950852573043e674f1744a023ba079
│   └── 24ea2234316139760800fc01a2a7e33752238d28c981627aa516e1c7d6cdfc34
│       └── ('miner', 33.333)
└── 0fa79e6746beb52eb3712714843347597c4cb5ebd71b1e07ca824a6fe1867cd3
    ├── 36f3e7f99d5006a15779c4bcff31f0c36915cdf2c800edd8d328fac0646955ad
    │   └── ('Bob', 'Bob', 5)
    └── e14b314e03bfd95c26374ac8a3837b39322c26e68ab1e72a2157506aabf6acd2
        └── ('Bob', 'miner', 1)


In [18]:
bc.wallets_taking_only_mined_blocks()

{'miner': 164.333, 'Alice': 10, 'Bob': 9}


In [19]:
print(f.block_newer_than(bc.chain[-2],bc.chain[-3]))
print(f.block_newer_than(bc.chain[-4],bc.chain[-2]))

True
False


In [20]:
#error para wallet inexistente
bc.add_transaction(bc.wallets,("Alice1","Bob",5))

TypeError: add_transaction() takes 2 positional arguments but 3 were given